In [72]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [73]:
credit_card_balance = pd.read_csv('credit_card_balance.csv')
#previous_application = pd.read_csv('previous_application.csv')

In [74]:
#Encoding categorical variables
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
def encoding_cat_vars(df,le):
    le_count = 0
    one_hot_count = 0
    # Iterate through the columns
    for col in df:
        if df[col].dtype == 'object' or df[col].dtype =='bool' :
            # If 2 or fewer unique categories
            if len(list(df[col].unique())) <= 2:
                # Train on the training data
                le.fit(df[col])
                # Transform both training and testing data
                df[col] = le.transform(df[col])    
                # Keep track of how many columns were label encoded
                le_count += 1
            else:
                one_hot_count +=1
    print ("no of columns label encoded : "+str(le_count))
    #One Hot encoding
    df = pd.get_dummies(df)
    print ("no of columns one hot encoded encoded : "+str(one_hot_count))
    return df

In [75]:
## Missing values###########
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns
###############################

In [76]:
#Removing highly correlated field in TMP_APP_TRAIN & TEST
def cor_find(df,threshold):
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    return corr_matrix,upper,to_drop

In [77]:
#Check for missing values
missing_values = missing_values_table(credit_card_balance)
missing_values['column_name'] = missing_values.index
missing_values = missing_values.reset_index(drop = True)
missing_values

Your selected dataframe has 23 columns.
There are 9 columns that have missing values.


,Missing Values,% of Total Values,column_name
0,767988,20.0,AMT_PAYMENT_CURRENT
1,749816,19.5,AMT_DRAWINGS_ATM_CURRENT
2,749816,19.5,AMT_DRAWINGS_OTHER_CURRENT
3,749816,19.5,AMT_DRAWINGS_POS_CURRENT
4,749816,19.5,CNT_DRAWINGS_ATM_CURRENT
5,749816,19.5,CNT_DRAWINGS_OTHER_CURRENT
6,749816,19.5,CNT_DRAWINGS_POS_CURRENT
7,305236,7.9,AMT_INST_MIN_REGULARITY
8,305236,7.9,CNT_INSTALMENT_MATURE_CUM


#### Imputing ATM DRAWINGS & CNT DRAWINGS . POS + OTHER + CURRENT = TOTAL

In [78]:
credit_card_balance['AMT_DRAWINGS_ATM_CURRENT'] = np.where(credit_card_balance['AMT_DRAWINGS_ATM_CURRENT'].isnull(),
                                                          np.where(credit_card_balance['AMT_DRAWINGS_CURRENT']==0,0,credit_card_balance['AMT_DRAWINGS_ATM_CURRENT']),
                                                          credit_card_balance['AMT_DRAWINGS_ATM_CURRENT'])
credit_card_balance['AMT_DRAWINGS_OTHER_CURRENT'] = np.where(credit_card_balance['AMT_DRAWINGS_OTHER_CURRENT'].isnull(),
                                                          np.where(credit_card_balance['AMT_DRAWINGS_CURRENT']==0,0,credit_card_balance['AMT_DRAWINGS_OTHER_CURRENT']),
                                                            credit_card_balance['AMT_DRAWINGS_OTHER_CURRENT'])
credit_card_balance['AMT_DRAWINGS_POS_CURRENT'] = np.where(credit_card_balance['AMT_DRAWINGS_POS_CURRENT'].isnull(),
                                                          np.where(credit_card_balance['AMT_DRAWINGS_CURRENT']==0,0,credit_card_balance['AMT_DRAWINGS_POS_CURRENT']),
                                                          credit_card_balance['AMT_DRAWINGS_POS_CURRENT'])

In [79]:
credit_card_balance['CNT_DRAWINGS_ATM_CURRENT'] = np.where(credit_card_balance['CNT_DRAWINGS_ATM_CURRENT'].isnull(),
                                                          np.where(credit_card_balance['CNT_DRAWINGS_CURRENT']==0,0,credit_card_balance['CNT_DRAWINGS_ATM_CURRENT']),
                                                          credit_card_balance['CNT_DRAWINGS_ATM_CURRENT'])
credit_card_balance['CNT_DRAWINGS_OTHER_CURRENT'] = np.where(credit_card_balance['CNT_DRAWINGS_OTHER_CURRENT'].isnull(),
                                                          np.where(credit_card_balance['CNT_DRAWINGS_CURRENT']==0,0,credit_card_balance['CNT_DRAWINGS_OTHER_CURRENT']),
                                                            credit_card_balance['CNT_DRAWINGS_OTHER_CURRENT'])
credit_card_balance['CNT_DRAWINGS_POS_CURRENT'] = np.where(credit_card_balance['CNT_DRAWINGS_POS_CURRENT'].isnull(),
                                                          np.where(credit_card_balance['CNT_DRAWINGS_CURRENT']==0,0,credit_card_balance['CNT_DRAWINGS_POS_CURRENT']),
                                                          credit_card_balance['CNT_DRAWINGS_POS_CURRENT'])

In [80]:
credit_card_balance['AMT_PAYMENT_CURRENT'] = credit_card_balance['AMT_PAYMENT_CURRENT'].fillna(0)

In [81]:
credit_card_balance[credit_card_balance['AMT_INST_MIN_REGULARITY'].isnull()]['AMT_DRAWINGS_CURRENT'].unique()

array([     0.  , 133380.  ,  58500.  , ...,  10282.5 , 108405.  ,
       184518.99])

In [82]:
len(credit_card_balance[credit_card_balance['AMT_INST_MIN_REGULARITY'] == credit_card_balance['AMT_PAYMENT_CURRENT']])

1267584

In [83]:
credit_card_balance['AMT_INST_MIN_REGULARITY'] = credit_card_balance['AMT_INST_MIN_REGULARITY'].fillna(credit_card_balance['AMT_PAYMENT_CURRENT'])

In [84]:
credit_card_balance['CNT_INSTALMENT_MATURE_CUM'] = credit_card_balance['CNT_INSTALMENT_MATURE_CUM'].fillna(0)

In [85]:
credit_card_balance[(credit_card_balance['AMT_DRAWINGS_ATM_CURRENT'] > credit_card_balance['AMT_DRAWINGS_CURRENT'])]

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
438776,2321147,198229,-64,0.0,112500,0.0,-1687.50,0.0,0.0,0.00,220.770,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,38.0,Active,0,0
747302,2340075,114867,-73,1718.1,180000,0.0,-519.57,0.0,0.0,9000.00,182340.000,182340.0,0.0,0.0,0.0,0.0,0,0.0,0.0,18.0,Active,0,0
3284667,2592574,217360,-96,0.0,270000,0.0,-6211.62,0.0,0.0,192.33,14672.205,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,7.0,Active,0,0


In [86]:
#Take status count
credit_card_status_count = credit_card_balance[['SK_ID_CURR','NAME_CONTRACT_STATUS']]
credit_card_status_count = credit_card_status_count.rename(columns={'NAME_CONTRACT_STATUS':'TOTAL_STATUS'})
credit_card_status_count = encoding_cat_vars(credit_card_status_count,le)
credit_card_status_count = credit_card_status_count.groupby(['SK_ID_CURR']).sum().reset_index()

no of columns label encoded : 0
no of columns one hot encoded encoded : 1


In [87]:
credit_card_status_count.head()

,SK_ID_CURR,TOTAL_STATUS_Active,TOTAL_STATUS_Approved,TOTAL_STATUS_Completed,TOTAL_STATUS_Demand,TOTAL_STATUS_Refused,TOTAL_STATUS_Sent proposal,TOTAL_STATUS_Signed
0,100006,6,0,0,0,0,0,0
1,100011,74,0,0,0,0,0,0
2,100013,96,0,0,0,0,0,0
3,100021,7,0,10,0,0,0,0
4,100023,8,0,0,0,0,0,0


In [88]:
#Fetch latest record
credit_card_month = credit_card_balance[['SK_ID_CURR','MONTHS_BALANCE']]
credit_card_latest = credit_card_balance[['SK_ID_CURR','MONTHS_BALANCE','AMT_BALANCE',
                                          'AMT_CREDIT_LIMIT_ACTUAL','AMT_DRAWINGS_ATM_CURRENT','AMT_DRAWINGS_OTHER_CURRENT',
                                          'AMT_DRAWINGS_POS_CURRENT','AMT_PAYMENT_CURRENT',
                                         'CNT_DRAWINGS_ATM_CURRENT','CNT_DRAWINGS_CURRENT','CNT_DRAWINGS_OTHER_CURRENT',
                                         'CNT_INSTALMENT_MATURE_CUM','SK_DPD','SK_DPD_DEF']]
credit_card_month = credit_card_month.groupby(['SK_ID_CURR']).max().reset_index()
credit_card_latest = pd.merge(credit_card_month,credit_card_latest,on=['SK_ID_CURR','MONTHS_BALANCE'],how='inner')

In [89]:
credit_card_latest = credit_card_latest.add_prefix('LATEST_').rename(columns={'LATEST_SK_ID_CURR':'SK_ID_CURR'})

In [90]:
credit_card_latest.head()

,SK_ID_CURR,LATEST_MONTHS_BALANCE,LATEST_AMT_BALANCE,LATEST_AMT_CREDIT_LIMIT_ACTUAL,LATEST_AMT_DRAWINGS_ATM_CURRENT,LATEST_AMT_DRAWINGS_OTHER_CURRENT,LATEST_AMT_DRAWINGS_POS_CURRENT,LATEST_AMT_PAYMENT_CURRENT,LATEST_CNT_DRAWINGS_ATM_CURRENT,LATEST_CNT_DRAWINGS_CURRENT,LATEST_CNT_DRAWINGS_OTHER_CURRENT,LATEST_CNT_INSTALMENT_MATURE_CUM,LATEST_SK_DPD,LATEST_SK_DPD_DEF
0,100006,-1,0.0,270000,0.0,0.0,0.0,0.000,0.0,0,0.0,0.0,0,0
1,100011,-2,0.0,90000,0.0,0.0,0.0,563.355,0.0,0,0.0,33.0,0,0
2,100013,-1,0.0,45000,0.0,0.0,0.0,274.320,0.0,0,0.0,22.0,0,0
3,100021,-2,0.0,675000,0.0,0.0,0.0,0.000,0.0,0,0.0,0.0,0,0
4,100023,-4,0.0,225000,0.0,0.0,0.0,0.000,0.0,0,0.0,0.0,0,0


In [91]:
credit_card_app_count = credit_card_balance[['SK_ID_CURR','SK_ID_PREV']]
credit_card_app_count = credit_card_app_count.rename(columns={'SK_ID_PREV':'CNT_CC_PREV_APP'})
credit_card_app_count = credit_card_app_count.groupby(['SK_ID_CURR'])['CNT_CC_PREV_APP'].nunique().reset_index()

In [92]:
credit_card_app_count.shape

(103558, 2)

In [93]:
credit_card_app_count.head()

,SK_ID_CURR,CNT_CC_PREV_APP
0,100006,1
1,100011,1
2,100013,1
3,100021,1
4,100023,1


### Synthesize new features

In [94]:
credit_card_balance['AMT_DRAWINGS_CURRENT / CNT_DRAWINGS_CURRENT'] =  credit_card_balance['AMT_DRAWINGS_CURRENT'] / (1 + credit_card_balance['CNT_DRAWINGS_CURRENT'])
credit_card_balance['AMT_DRAWINGS_ATM_CURRENT / AMT_DRAWINGS_CURRENT'] =  credit_card_balance['AMT_DRAWINGS_ATM_CURRENT'] / (1 + credit_card_balance['AMT_DRAWINGS_CURRENT'])
credit_card_balance['CNT_DRAWINGS_ATM_CURRENT / CNT_DRAWINGS_CURRENT'] =  credit_card_balance['CNT_DRAWINGS_ATM_CURRENT'] / (1 + credit_card_balance['CNT_DRAWINGS_CURRENT'])
credit_card_balance['AMT_INST_MIN_REGULARITY / AMT_PAYMENT_TOTAL_CURRENT' ] = credit_card_balance['AMT_INST_MIN_REGULARITY'] / (1 + credit_card_balance['AMT_PAYMENT_TOTAL_CURRENT'])
credit_card_balance['AMT_DRAWINGS_CURRENT / AMT_CREDIT_LIMIT_ACTUAL' ] =  credit_card_balance['AMT_DRAWINGS_CURRENT'] / (1 + credit_card_balance['AMT_CREDIT_LIMIT_ACTUAL'])
credit_card_balance['AMT_BALANCE / AMT_CREDIT_LIMIT_ACTUAL' ] =  credit_card_balance['AMT_BALANCE'] / (1 + credit_card_balance['AMT_CREDIT_LIMIT_ACTUAL'])

### FInd default rate of each statuses!!

In [95]:
app_train = pd.read_csv('application_train.csv')[['SK_ID_CURR','TARGET']]

In [96]:
credit_card_statusrate = pd.merge(credit_card_balance[['SK_ID_CURR','NAME_CONTRACT_STATUS']],app_train,on=['SK_ID_CURR'],how='left')

In [97]:
cc_rate = credit_card_statusrate.groupby(['NAME_CONTRACT_STATUS','TARGET'])[['SK_ID_CURR']].count().reset_index()
cc_rate = cc_rate.pivot(index='NAME_CONTRACT_STATUS', columns='TARGET', values='SK_ID_CURR').reset_index()
cc_rate = cc_rate.rename(columns={0:'CC_STATUS_CORRECT',1:'CC_STATUS_DEFAULT'})

In [98]:
cc_rate = cc_rate.fillna(0)
cc_rate

TARGET,NAME_CONTRACT_STATUS,CC_STATUS_CORRECT,CC_STATUS_DEFAULT
0,Active,2893941.0,222732.0
1,Approved,5.0,0.0
2,Completed,93611.0,6420.0
3,Demand,675.0,168.0
4,Refused,12.0,2.0
5,Sent proposal,442.0,18.0
6,Signed,9278.0,661.0


In [99]:
credit_card_statusrate = pd.merge(credit_card_statusrate,cc_rate,on=['NAME_CONTRACT_STATUS'],how='left')
credit_card_statusrate['DEFAULT_RATE_CC_STATUS'] = credit_card_statusrate['CC_STATUS_DEFAULT']/credit_card_statusrate['CC_STATUS_CORRECT']
credit_card_statusrate = credit_card_statusrate.drop(columns=['CC_STATUS_DEFAULT','CC_STATUS_CORRECT'])

In [100]:
credit_card_statusrate = credit_card_statusrate.drop(columns=['TARGET'])

In [101]:
credit_card_statusrate = credit_card_statusrate.drop(columns=['NAME_CONTRACT_STATUS'])

In [102]:
credit_card_statusrate = credit_card_statusrate.groupby(['SK_ID_CURR']).mean().reset_index()

In [103]:
credit_card_statusrate.head()

,SK_ID_CURR,DEFAULT_RATE_CC_STATUS
0,100006,0.076965
1,100011,0.076965
2,100013,0.076965
3,100021,0.072034
4,100023,0.076965


In [104]:
credit_card_stat = credit_card_balance[['SK_ID_CURR','SK_ID_PREV','NAME_CONTRACT_STATUS','SK_DPD','SK_DPD_DEF']]

In [105]:
credit_card_stat_dpd = credit_card_stat.groupby(['SK_ID_CURR','SK_ID_PREV'])[['SK_DPD','SK_DPD_DEF']].max().reset_index()

In [40]:
credit_card_stat_dpd = credit_card_stat_dpd.drop(columns=['SK_ID_PREV'])

In [41]:
credit_card_stat_dpd.head()

,SK_ID_CURR,SK_DPD,SK_DPD_DEF
0,100006,0,0
1,100011,0,0
2,100013,1,1
3,100021,0,0
4,100023,0,0


In [42]:
credit_card_stat_dpd = credit_card_stat_dpd.groupby(['SK_ID_CURR']).sum().reset_index()

In [63]:
credit_card_stat_dpd.head()

,SK_ID_CURR,SK_DPD,SK_DPD_DEF
0,100006,0,0
1,100011,0,0
2,100013,1,1
3,100021,0,0
4,100023,0,0


In [45]:
credit_card_stat = credit_card_stat.groupby(['SK_ID_CURR','SK_ID_PREV'])['NAME_CONTRACT_STATUS'].first().reset_index()
credit_card_stat = credit_card_stat.drop(columns=['SK_ID_PREV'])
credit_card_stat = encoding_cat_vars(credit_card_stat,le)

no of columns label encoded : 0
no of columns one hot encoded encoded : 1


In [46]:
credit_card_stat.head()

,SK_ID_CURR,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Signed
0,100006,1,0,0,0,0
1,100011,1,0,0,0,0
2,100013,1,0,0,0,0
3,100021,0,1,0,0,0
4,100023,1,0,0,0,0


In [47]:
credit_card_stat = credit_card_stat.groupby(['SK_ID_CURR']).sum().reset_index()

In [48]:
credit_card_stat.head()

,SK_ID_CURR,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Signed
0,100006,1,0,0,0,0
1,100011,1,0,0,0,0
2,100013,1,0,0,0,0
3,100021,0,1,0,0,0
4,100023,1,0,0,0,0


In [49]:
credit_card_stat = pd.merge(credit_card_stat,credit_card_stat_dpd,on=['SK_ID_CURR'],how='inner')

In [51]:
credit_card_stat.head()

,SK_ID_CURR,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Signed,SK_DPD,SK_DPD_DEF
0,100006,1,0,0,0,0,0,0
1,100011,1,0,0,0,0,0,0
2,100013,1,0,0,0,0,1,1
3,100021,0,1,0,0,0,0,0
4,100023,1,0,0,0,0,0,0


In [52]:
cc_derived_features = credit_card_balance[['SK_ID_CURR','AMT_DRAWINGS_CURRENT / CNT_DRAWINGS_CURRENT',
                                          'AMT_INST_MIN_REGULARITY / AMT_PAYMENT_TOTAL_CURRENT',
                                          'AMT_DRAWINGS_CURRENT / AMT_CREDIT_LIMIT_ACTUAL',
                                          'AMT_BALANCE / AMT_CREDIT_LIMIT_ACTUAL','AMT_DRAWINGS_ATM_CURRENT / AMT_DRAWINGS_CURRENT',
                                          'CNT_DRAWINGS_ATM_CURRENT / CNT_DRAWINGS_CURRENT']]

In [53]:
cc_derived_features_sum = cc_derived_features.add_prefix('SUM_').rename(columns={'SUM_SK_ID_CURR':'SK_ID_CURR'})
cc_derived_features_max = cc_derived_features.add_prefix('MAX_').rename(columns={'MAX_SK_ID_CURR':'SK_ID_CURR'})
cc_derived_features_mean = cc_derived_features.add_prefix('MEAN_').rename(columns={'MEAN_SK_ID_CURR':'SK_ID_CURR'})

cc_derived_features_sum = cc_derived_features_sum.groupby(['SK_ID_CURR']).sum().reset_index()
cc_derived_features_max = cc_derived_features_max.groupby(['SK_ID_CURR']).max().reset_index()
cc_derived_features_mean = cc_derived_features_mean.groupby(['SK_ID_CURR']).mean().reset_index()

In [54]:
cc_derived_features = pd.merge(cc_derived_features_sum,cc_derived_features_mean,on=['SK_ID_CURR'],how='inner')
cc_derived_features = pd.merge(cc_derived_features,cc_derived_features_max,on=['SK_ID_CURR'],how='inner')
cc_derived_features = pd.merge(cc_derived_features,credit_card_statusrate,on=['SK_ID_CURR'],how='inner')


In [55]:
cc_derived_features.head()

,SK_ID_CURR,SUM_AMT_DRAWINGS_CURRENT / CNT_DRAWINGS_CURRENT,SUM_AMT_INST_MIN_REGULARITY / AMT_PAYMENT_TOTAL_CURRENT,SUM_AMT_DRAWINGS_CURRENT / AMT_CREDIT_LIMIT_ACTUAL,SUM_AMT_BALANCE / AMT_CREDIT_LIMIT_ACTUAL,SUM_AMT_DRAWINGS_ATM_CURRENT / AMT_DRAWINGS_CURRENT,SUM_CNT_DRAWINGS_ATM_CURRENT / CNT_DRAWINGS_CURRENT,MEAN_AMT_DRAWINGS_CURRENT / CNT_DRAWINGS_CURRENT,MEAN_AMT_INST_MIN_REGULARITY / AMT_PAYMENT_TOTAL_CURRENT,MEAN_AMT_DRAWINGS_CURRENT / AMT_CREDIT_LIMIT_ACTUAL,MEAN_AMT_BALANCE / AMT_CREDIT_LIMIT_ACTUAL,MEAN_AMT_DRAWINGS_ATM_CURRENT / AMT_DRAWINGS_CURRENT,MEAN_CNT_DRAWINGS_ATM_CURRENT / CNT_DRAWINGS_CURRENT,MAX_AMT_DRAWINGS_CURRENT / CNT_DRAWINGS_CURRENT,MAX_AMT_INST_MIN_REGULARITY / AMT_PAYMENT_TOTAL_CURRENT,MAX_AMT_DRAWINGS_CURRENT / AMT_CREDIT_LIMIT_ACTUAL,MAX_AMT_BALANCE / AMT_CREDIT_LIMIT_ACTUAL,MAX_AMT_DRAWINGS_ATM_CURRENT / AMT_DRAWINGS_CURRENT,MAX_CNT_DRAWINGS_ATM_CURRENT / CNT_DRAWINGS_CURRENT,DEFAULT_RATE_CC_STATUS
0,100006,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.000,0.076965
1,100011,36000.0,835.353759,0.999994,22.398077,0.999994,0.80,486.486486,11.288564,0.013513,0.302677,0.013513,0.010811,36000.0,804.195,0.999994,1.049994,0.999994,0.800,0.076965
2,100013,99450.0,11333.783339,3.628548,11.068833,4.999926,3.85,1035.937500,118.060243,0.037797,0.115300,0.052083,0.040104,29700.0,7875.000,0.999994,1.024884,0.999994,0.875,0.076965
3,100021,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.000,0.072034
4,100023,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.000,0.076965


In [56]:
credit_card_cal =  credit_card_balance[['SK_ID_CURR','MONTHS_BALANCE','AMT_BALANCE','AMT_CREDIT_LIMIT_ACTUAL',
                                       'AMT_DRAWINGS_ATM_CURRENT','AMT_DRAWINGS_CURRENT','AMT_DRAWINGS_OTHER_CURRENT',
                                       'AMT_DRAWINGS_POS_CURRENT','AMT_INST_MIN_REGULARITY','AMT_PAYMENT_CURRENT',
                                       'AMT_PAYMENT_TOTAL_CURRENT','AMT_PAYMENT_TOTAL_CURRENT','AMT_RECEIVABLE_PRINCIPAL',
                                       'AMT_RECIVABLE','AMT_TOTAL_RECEIVABLE','CNT_DRAWINGS_ATM_CURRENT','CNT_DRAWINGS_CURRENT',
                                       'CNT_DRAWINGS_OTHER_CURRENT','CNT_DRAWINGS_POS_CURRENT','CNT_INSTALMENT_MATURE_CUM']]
credit_card_mean = credit_card_cal.add_prefix('MEAN_').rename(columns={'MEAN_SK_ID_CURR':'SK_ID_CURR'})
credit_card_max = credit_card_cal.add_prefix('MAX_').rename(columns={'MAX_SK_ID_CURR':'SK_ID_CURR'})
credit_card_min = credit_card_cal.add_prefix('MIN_').rename(columns={'MIN_SK_ID_CURR':'SK_ID_CURR'})
credit_card_sum = credit_card_cal.add_prefix('SUM_').rename(columns={'SUM_SK_ID_CURR':'SK_ID_CURR'})
credit_card_var = credit_card_cal.add_prefix('VAR_').rename(columns={'VAR_SK_ID_CURR':'SK_ID_CURR'})

In [57]:
credit_card_mean = credit_card_mean.groupby(['SK_ID_CURR']).mean().reset_index()
credit_card_max = credit_card_max.groupby(['SK_ID_CURR']).max().reset_index()
credit_card_min = credit_card_min.groupby(['SK_ID_CURR']).min().reset_index()
credit_card_sum = credit_card_sum.groupby(['SK_ID_CURR']).sum().reset_index()
credit_card_var = credit_card_var.groupby(['SK_ID_CURR']).var().reset_index()

In [58]:
#Fetch latest per customer
credit_car_per_cust = credit_card_latest.add_prefix('LATEST_').rename(columns={'LATEST_SK_ID_CURR':'SK_ID_CURR'})
credit_car_per_cust_max = credit_car_per_cust.groupby(['SK_ID_CURR']).max().reset_index()
credit_car_per_cust_sum = credit_car_per_cust.groupby(['SK_ID_CURR']).max().reset_index()
credit_car_per_cust = pd.merge(credit_car_per_cust_max,credit_car_per_cust_sum,on=['SK_ID_CURR'],how='inner')

In [59]:
credit_car_per_cust['SK_ID_CURR'].nunique()

103558

In [60]:
#Calculate total DPD per customer
credit_card_dpd = credit_card_balance[['SK_ID_CURR','SK_DPD_DEF','SK_DPD']]
credit_card_dpd = credit_card_dpd.groupby(['SK_ID_CURR']).sum().reset_index()
credit_card_dpd = credit_card_dpd.rename(columns={'SK_DPD_DEF':'CC_SK_DPD_DEF_COUNT','SK_DPD':'CC_SK_DPD_COUNT'})

In [61]:
credit_card_dpd.head()

,SK_ID_CURR,CC_SK_DPD_DEF_COUNT,CC_SK_DPD_COUNT
0,100006,0,0
1,100011,0,0
2,100013,1,1
3,100021,0,0
4,100023,0,0


In [62]:
credit_card_stat.head()

,SK_ID_CURR,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Signed,SK_DPD,SK_DPD_DEF
0,100006,1,0,0,0,0,0,0
1,100011,1,0,0,0,0,0,0
2,100013,1,0,0,0,0,1,1
3,100021,0,1,0,0,0,0,0
4,100023,1,0,0,0,0,0,0


In [64]:
#Joining with status
credit_car_per_cust = pd.merge(credit_car_per_cust,credit_card_stat,how='left',on=['SK_ID_CURR'])
#credit_card_latest = pd.merge(credit_card_latest,credit_card_status_count,how='left',on=['SK_ID_CURR'])
credit_car_per_cust = pd.merge(credit_car_per_cust,credit_card_statusrate,how='left',on=['SK_ID_CURR']) 
#Join with DPD
credit_car_per_cust = pd.merge(credit_car_per_cust,credit_card_dpd,how='left',on=['SK_ID_CURR'])

In [65]:
credit_car_per_cust = pd.merge(credit_car_per_cust,credit_card_app_count,how='inner',on=['SK_ID_CURR'])
credit_car_per_cust = pd.merge(credit_car_per_cust,credit_card_mean,how='inner',on=['SK_ID_CURR'])
credit_car_per_cust = pd.merge(credit_car_per_cust,credit_card_max,how='inner',on=['SK_ID_CURR'])
credit_car_per_cust = pd.merge(credit_car_per_cust,credit_card_min,how='inner',on=['SK_ID_CURR'])
credit_car_per_cust = pd.merge(credit_car_per_cust,credit_card_sum,how='inner',on=['SK_ID_CURR'])
credit_car_per_cust = pd.merge(credit_car_per_cust,credit_card_var,how='inner',on=['SK_ID_CURR'])

In [66]:
credit_car_per_cust = pd.merge(credit_car_per_cust,cc_derived_features,how='inner',on=['SK_ID_CURR'])

In [67]:
credit_car_per_cust.shape

(103558, 152)

In [68]:
#Check for missing values
missing_values = missing_values_table(credit_car_per_cust)
missing_values['column_name'] = missing_values.index
missing_values = missing_values.reset_index(drop = True)
missing_values

Your selected dataframe has 152 columns.
There are 19 columns that have missing values.


,Missing Values,% of Total Values,column_name
0,692,0.7,VAR_MONTHS_BALANCE
1,692,0.7,VAR_AMT_PAYMENT_TOTAL_CURRENT
2,692,0.7,VAR_CNT_DRAWINGS_POS_CURRENT
3,692,0.7,VAR_CNT_DRAWINGS_OTHER_CURRENT
4,692,0.7,VAR_CNT_DRAWINGS_CURRENT
5,692,0.7,VAR_CNT_DRAWINGS_ATM_CURRENT
6,692,0.7,VAR_AMT_TOTAL_RECEIVABLE
7,692,0.7,VAR_AMT_RECIVABLE
8,692,0.7,VAR_AMT_RECEIVABLE_PRINCIPAL
9,692,0.7,VAR_AMT_PAYMENT_TOTAL_CURRENT


In [ ]:
# del_cols = missing_values[missing_values['% of Total Values']>30]['column_name']
# del_cols
# credit_car_per_cust = credit_car_per_cust.drop(columns=del_cols.values,axis = 1)

In [69]:
credit_car_per_cust = credit_car_per_cust.dropna()

In [ ]:
# #Fill NAN with a value not present in data set ¬ -10
# credit_car_per_cust = credit_car_per_cust.fillna(-1)

In [70]:
credit_car_per_cust.shape

(102866, 152)

In [ ]:
# credit_car_per_cust = encoding_cat_vars(credit_car_per_cust,le)

In [71]:
credit_car_per_cust.to_csv('tmp_credit_card.csv',index=False)